In [3]:
%cd swoop-machine-learning-v2

/home/studio-lab-user/swoop-machine-learning-v2


In [11]:
# import ai_functions
# cannot import boto3, sklearn, utils definitions
import ai_functions

In [1]:
ENVIRONMENT = 'testing'
AI_BUCKET = "snapdragon-{}-ai".format(ENVIRONMENT) #definitions.py
# s3_client = boto3.client("s3")
# s3_bucket = boto3.resource("s3").Bucket(AI_BUCKET)

In [2]:
# inside keras?
MODEL_WEIGHTS_LOCATION = ".keras/VGG16_weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [3]:
# When calling ImageModel, need image_json, data, product_id, result_key, can_test, logger arguments needed
class ImageModel:
    def __init__(self, image_json, data, product_id, result_key, can_test, logger):
        self.image_json = image_json
        self.product_id = product_id
        self.result_key = result_key
        self.prediction_data = data
        self.test_data = data[data["image"].notna()]
        self.can_test = can_test,
        self.hash_lists = {} # fill image hash lists
        self.tmp_dir = None
        self.dir_lists = {}
        self.model = None
        # from keras.preprocessing.image import ImageDataGenerator
        # constants from image_model_constants.py. It is in utils folder.
        self.train_datagen = ImageDataGenerator(
            rescale=1. / 255,  # rescales pixel values
            rotation_range=constants.ROTATION_RANGE,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range=constants.ZOOM_RANGE, # Randomly zoom image
            width_shift_range=constants.WIDTH_SHIFT_RANGE,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=constants.HEIGHT_SHIFT_RANGE,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=constants.HORIZONTAL_FLIP,  # randomly flip images
            vertical_flip=constants.VERTICAL_FLIP,  # randomly flip images
        )
        self.val_datagen = ImageDataGenerator(rescale=1. / 255)
        self.test_datagen = ImageDataGenerator(rescale=1. / 255)
        self.predict_datagen = ImageDataGenerator(rescale=1. / 255)
        self.generators = {}
        self.model_details = {}
        self.test_probabilities = None
        self.test_results = pd.DataFrame()
        self.roc_auc = None
        self.prediction_probabilities = None
        self.prediction_results = pd.DataFrame(columns=["image_score", "image"])
        self.prediction_generator_filenames = None
        self.logger = logger

In [4]:
# get_image_hash_lists. self.hash_lists is defined empty {}
# wonder what's the output of image_json["infringing_hashes"]
def set_image_hash_lists(self):
    self.hash_lists["infringing_hashes"] = self.image_json["infringing_hashes"]
    self.hash_lists["innocent_hashes"] = self.image_json["innocent_hashes"]
    self.hash_lists["irrelevant_hashes"] = self.image_json["irrelevant_hashes"]
    self.logger.info("Image training hash lists compiled")

In [5]:
def create_training_temporary_directory_structure(self):
    self.tmp_dir = tempfile.mkdtemp()

    os.mkdir(self.tmp_dir + "/train_dir")
    os.mkdir(self.tmp_dir + "/train_dir/infringing")
    os.mkdir(self.tmp_dir + "/train_dir/innocent")
    os.mkdir(self.tmp_dir + "/train_dir/irrelevant")

    os.mkdir(self.tmp_dir + "/val_dir")
    os.mkdir(self.tmp_dir + "/val_dir/infringing")
    os.mkdir(self.tmp_dir + "/val_dir/innocent")
    os.mkdir(self.tmp_dir + "/val_dir/irrelevant")

    os.mkdir(self.tmp_dir + "/test_dir")
    os.mkdir(self.tmp_dir + "/test_dir/infringing")
    os.mkdir(self.tmp_dir + "/test_dir/innocent")
    os.mkdir(self.tmp_dir + "/test_dir/irrelevant")

    os.mkdir(self.tmp_dir + "/keras_model")
    os.mkdir(self.tmp_dir + "/keras_model/variables")
    os.mkdir(self.tmp_dir + "/keras_model/assets")

    self.logger.info("Training temporary directories created")

In [6]:
def shuffle_image_lists(self):
    random.shuffle(self.hash_lists["infringing_hashes"])
    random.shuffle(self.hash_lists["innocent_hashes"])
    random.shuffle(self.hash_lists["irrelevant_hashes"])

In [7]:
# class_label, required_image_count, excluded_images should be included as arguments
# self.test_data = data[data["image"].notna()] == class_label. What is ["class"] column? [0,1,2] [infringing, innocent, irrlevant]
# self.prediction_data = data

def get_lists_for_test_directory(self, class_label, required_image_count, excluded_images):
    dataframe = self.test_data[self.test_data["class"] == class_label]
    # except ['iamge'] column, isin(exclude_images) & "empty"
    dataframe = dataframe[~dataframe["image"].isin(excluded_images)]
    dataframe = dataframe[~dataframe["image"].isin(["empty"])]
    
    # randomly sample with minimum number of required_image_count
    # return dataframe['image'] as a list.
    dataframe = dataframe.sample(n=min(required_image_count, len(dataframe)))
    return dataframe["image"].tolist()

In [32]:
# in dir_lists {}, adding train, val infringing, innocent, irrelevant hashes.
# self.dir_lists is an empty {}. Why not empty [] but {}? Each dictionary A,B,C,D can contain 8-12 items (lists). This is neat and convenient for the furue use.
# hash_lists["infringing_hashes"][:int(len*0.75)]. Why :int? because len*0.75 can be float. Make it int and slicing.
def get_lists_for_each_directory(self):
    # TRAIN_VAL_SPLIT = 0.75 in constants.py
        self.dir_lists["train_infringing_hashes"] = self.hash_lists["infringing_hashes"][:int(
            len(self.hash_lists["infringing_hashes"]) * constants.TRAIN_VAL_SPLIT)]
        self.dir_lists["val_infringing_hashes"] = self.hash_lists["infringing_hashes"][int(
            len(self.hash_lists["infringing_hashes"]) * constants.TRAIN_VAL_SPLIT):]

        self.dir_lists["train_innocent_hashes"] = self.hash_lists["innocent_hashes"][:int(
            len(self.hash_lists["innocent_hashes"]) * constants.TRAIN_VAL_SPLIT)]
        self.dir_lists["val_innocent_hashes"] = self.hash_lists["innocent_hashes"][int(
            len(self.hash_lists["innocent_hashes"]) * constants.TRAIN_VAL_SPLIT):]

        self.dir_lists["train_irrelevant_hashes"] = self.hash_lists["irrelevant_hashes"][:int(
            len(self.hash_lists["irrelevant_hashes"]) * constants.TRAIN_VAL_SPLIT)]
        self.dir_lists["val_irrelevant_hashes"] = self.hash_lists["irrelevant_hashes"][int(
            len(self.hash_lists["irrelevant_hashes"]) * constants.TRAIN_VAL_SPLIT):]
# test_infringing_hashes = get_list_directory(class_label = 'infringing', required_image_count = 40,40,200 (im_constants.py), excluded_images = self.hash_lists["infringing_hashes"])
        self.dir_lists["test_infringing_hashes"] = self.get_lists_for_test_directory(
            "infringing", constants.INFRINGING_TEST_IMAGE_COUNT, self.hash_lists["infringing_hashes"])
        self.dir_lists["test_innocent_hashes"] = self.get_lists_for_test_directory(
            "innocent", constants.INNOCENT_TEST_IMAGE_COUNT, self.hash_lists["innocent_hashes"])
        self.dir_lists["test_irrelevant_hashes"] = self.get_lists_for_test_directory(
            "irrelevant", constants.IRRELEVANT_TEST_IMAGE_COUNT, self.hash_lists["irrelevant_hashes"])


In [33]:
def add_images_to_directories(self):
        # running a method - shuffling image lists
        self.shuffle_image_lists()
        # running a method - split_train_val
        self.get_lists_for_each_directory()
        # download training_infringing images. arguments - hash_list = train_infrining_hashes, path = train_dir/infringing 
        ai_functions.parallel_download_images_from_s3(
                                self.dir_lists["train_infringing_hashes"],
                                (self.tmp_dir + "/train_dir/infringing"))
        # download validating_infringing images. arguments - hash_list = val_infrining_hashes, path = val_dir/infringing 
        ai_functions.parallel_download_images_from_s3(
                                self.dir_lists["val_infringing_hashes"],
                                (self.tmp_dir + "/val_dir/infringing"))
        # download testing_infringing images. 
        if len(self.dir_lists["test_infringing_hashes"]) > 0:
            ai_functions.parallel_download_images_from_s3(
                                    self.dir_lists["test_infringing_hashes"],
                                    (self.tmp_dir + "/test_dir/infringing"))
        # download training_innocent images.
        ai_functions.parallel_download_images_from_s3(
                                self.dir_lists["train_innocent_hashes"],
                                (self.tmp_dir + "/train_dir/innocent"))
        # download validating_innocent images.
        ai_functions.parallel_download_images_from_s3(
                                self.dir_lists["val_innocent_hashes"],
                                (self.tmp_dir + "/val_dir/innocent"))
        # download teting_innocent images.
        if len(self.dir_lists["test_innocent_hashes"]) > 0:
            ai_functions.parallel_download_images_from_s3(
                                    self.dir_lists["test_innocent_hashes"],
                                    (self.tmp_dir + "/test_dir/innocent"))
        # download training_irrelevant images.
        ai_functions.parallel_download_images_from_s3(
                                self.dir_lists["train_irrelevant_hashes"],
                                (self.tmp_dir + "/train_dir/irrelevant"))
        # download validating_irrelevant images.
        ai_functions.parallel_download_images_from_s3(
                                self.dir_lists["val_irrelevant_hashes"],
                                (self.tmp_dir + "/val_dir/irrelevant"))
        # download testing_irrelevant images.
        if len(self.dir_lists["test_irrelevant_hashes"]) > 0:
            ai_functions.parallel_download_images_from_s3(
                                    self.dir_lists["test_irrelevant_hashes"],
                                    (self.tmp_dir + "/test_dir/irrelevant"))

        self.logger.info("Training images added to temporary directories")
        self.items_in_test_infringing = len([name for name in os.listdir(self.tmp_dir+"/test_dir/infringing/")])
        self.items_in_test_innocent = len([name for name in os.listdir(self.tmp_dir+"/test_dir/innocent/")])
        self.items_in_test_irrelevant = len([name for name in os.listdir(self.tmp_dir+"/test_dir/irrelevant/")])

        self.logger.info("items in test infringing directory = {}".format(self.items_in_test_infringing))
        self.logger.info("items in test innocent directory = {}".format(self.items_in_test_innocent))
        self.logger.info("items in test irrelevant directory = {}".format(self.items_in_test_irrelevant))

In [34]:
def check_training_directories_for_corrupt_images(self):
    directories = [self.tmp_dir + "/train_dir/infringing",
                   self.tmp_dir + "/val_dir/infringing",
                   self.tmp_dir + "/test_dir/infringing",
                   self.tmp_dir + "/train_dir/innocent",
                   self.tmp_dir + "/val_dir/innocent",
                   self.tmp_dir + "/test_dir/innocent",
                   self.tmp_dir + "/train_dir/irrelevant",
                   self.tmp_dir + "/val_dir/irrelevant",
                   self.tmp_dir + "/test_dir/irrelevant",
                   ]
    ai_functions.delete_corrupt_images(directories, self.logger)
    self.logger.info("Training directories checked for corrupt image files.")

In [8]:
# VGG(weights, include_top, input_she = (224, 224, 3) for layer in vgg_conv.layers[:]: layer.trainable=False
# DENSE_LAYER_SIZE = 1024
# DROPOUT_LEVEL = 0.5
def build_cnn_model(self):
    if self.model_type_info["classes"] == "two":
        final_layer_size = 2
    else:
        final_layer_size = 3

    vgg_conv = VGG16(
        # TODO: Use path once issue is fixed.
        # weights = MODEL_WEIGHTS_LOCATION,
        weights="imagenet",
        include_top=False,
        input_shape=(constants.IMAGE_SIZE, constants.IMAGE_SIZE, 3),
    )
    for layer in vgg_conv.layers[:]:
        layer.trainable = False

    self.model = Sequential()
    self.model.add(vgg_conv)
    self.model.add(Flatten())
    self.model.add(Dense(constants.DENSE_LAYER_SIZE, activation="relu"))
    self.model.add(Dropout(constants.DROPOUT_LEVEL))
    self.model.add(Dense(final_layer_size, activation="softmax"))

    self.logger.info("Neural Network built")


In [9]:
# ImageDataGenerator.train_datagenerator.flow_from_directory(data augmentation)
# BATCHSIZE = 10
def set_generator_directories(self):
        self.generators["train_generator"] = self.train_datagen.flow_from_directory(
            (self.tmp_dir + "/train_dir"),
            target_size=(constants.IMAGE_SIZE, constants.IMAGE_SIZE),
            batch_size=constants.BATCHSIZE,
            class_mode="categorical",
            shuffle=True,
        )

        self.generators["val_generator"] = self.val_datagen.flow_from_directory(
            (self.tmp_dir + "/val_dir"),
            target_size=(constants.IMAGE_SIZE, constants.IMAGE_SIZE),
            batch_size=constants.BATCHSIZE,
            class_mode="categorical",
            shuffle=False,
        )

        self.generators["test_generator"] = self.test_datagen.flow_from_directory(
            (self.tmp_dir + "/test_dir"),
            target_size=(constants.IMAGE_SIZE, constants.IMAGE_SIZE),
            batch_size=constants.BATCHSIZE,
            class_mode="categorical",
            shuffle=False,
        )

        self.logger.info("Data Generators set")


In [1]:
# EarlyStopping when validation loss is not reduced
# EARLY_STOPPING_PATIENCE = 7
# LEARNING_RATE_PATIENCE = 3
# LEARNING_RATE_VERBOSE = 1
# LEARNING_RATE_REDUCTION_FACTOR = 0.5
# LEARNING_RATE_MIN = 0.00001

def set_model_training_callbacks(self):
        self.model_details["early_stopping"] = EarlyStopping(
            monitor="val_loss",
            patience=constants.EARLY_STOPPING_PATIENCE,
        )

        self.model_details["learning_rate_reduction"] = ReduceLROnPlateau(
            monitor="val_loss",
            patience=constants.LEARNING_RATE_PATIENCE,
            verbose=constants.LEARNING_RATE_VERBOSE,
            factor=constants.LEARNING_RATE_REDUCTION_FACTOR,
            min_lr=constants.LEARNING_RATE_MIN,
        )

        self.logger.info("Model training callbacks set")


In [2]:
def compile_model(self):
        self.model.compile(
            loss="categorical_crossentropy",
            optimizer=optimizers.RMSprop(),
            metrics=["acc"],
        )
        self.logger.info("Model compiled")

In [3]:
def train_model(self):
    self.model_details["history"] = self.model.fit(
        self.generators["train_generator"],
        steps_per_epoch=self.generators["train_generator"].samples / self.generators["train_generator"].batch_size,
        epochs=constants.FITTING_EPOCHS,
        validation_data=self.generators["val_generator"],
        validation_steps=self.generators["val_generator"].samples / self.generators["val_generator"].batch_size,
        verbose=constants.FITTING_VERBOSE,
        callbacks=[self.model_details["learning_rate_reduction"], self.model_details["early_stopping"]],
    )
    self.logger.info("Model training completed")

In [4]:
def save_trained_model_to_temporary_directory(self):
    save_model(
        model = self.model,
        filepath = "{}/keras_model".format(self.tmp_dir),
        )
    self.logger.info("Trained model saved to temporary directory")

In [5]:
def save_trained_model_to_s3(self):
    save_keras_model_to_s3(s3_client, self.product_id, self.tmp_dir)
    self.logger.info("Trained model saved to S3")

In [7]:
def get_test_probabilities(self):
    self.test_probabilities = self.model.predict(
        x=self.generators["test_generator"],
        verbose=1)
    self.test_probabilities_df = pd.DataFrame(self.test_probabilities, columns = np.unique(self.test_data["class"]))

In [8]:
# INNOCENT_SCORE_FACTOR = 0.5
def calculate_image_test_score(self, row):
    score = 0
    score = score + row['infringing']
    if self.model_type_info["classes"] == "three":
        score = score + (row['innocent'] * INNOCENT_SCORE_FACTOR)
    return score